# RNN text sentiment analyzer

In [8]:
from data_processing.text_datasets import get_poem_sentiment_dataset
import torch
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
import torch.nn as nn
import tqdm
from torch.nn.utils.rnn import pad_sequence

In [61]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [9]:
corpus = api.load("text8")
gensim_model = Word2Vec(corpus)
tokenizer = gensim_model.wv.key_to_index
emb_weights = torch.FloatTensor(gensim_model.wv.vectors)


def tokenize_function(examples):
    review_tokenized = []
    all_parsed = 0
    unknows = 0
    for word in examples.split():
        all_parsed += 1
        try:
            review_tokenized.append(tokenizer[word.lower()])
        except:
            unknows += 1
    return review_tokenized

In [56]:
def pad_collate(batch):
    xx, yy = zip(*batch)
    x_lens = [len(x)-1 for x in xx]

    xx_pad = pad_sequence(xx, batch_first=True, padding_value=0)
    yy = torch.stack(yy)
    return xx_pad, yy, x_lens

In [57]:
train_dataset, validation_dataset, test_dataset = get_poem_sentiment_dataset(
    tokenize_function
)

validation_loader = torch.utils.data.DataLoader(
    validation_dataset, batch_size=32, shuffle=False, collate_fn=pad_collate
)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=pad_collate)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, collate_fn=pad_collate)

# Model

In [58]:
class LstmSentimentClasifier(nn.Module):
    def __init__(
        self,
        embedding_dim,
        hidden_size,
        num_of_rnn_layers,
        out_size,
        emb_weights,
        bidirectional=False,
    ):
        super().__init__()
        self.num_of_rnn_layers = num_of_rnn_layers
        self.hidden_size = hidden_size
        if bidirectional:
            self.bidirectional = 2
        else:
            self.bidirectional = 1
        self.embeddings = nn.Embedding.from_pretrained(emb_weights)
        self.embeddings.requires_grad = False
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            num_layers=num_of_rnn_layers,
            bidirectional=bidirectional,
            batch_first=False,
        )
        self.fc = nn.Linear(hidden_size * self.bidirectional, out_size)

    def init_hidden(self, batch_size):
        hidden = torch.zeros(
            self.num_of_rnn_layers * self.bidirectional, batch_size, self.hidden_size
        )
        state = torch.zeros(
            self.num_of_rnn_layers * self.bidirectional, batch_size, self.hidden_size
        )
        return hidden, state

    def forward(self, x, len_x, hidden):
        x = self.embeddings(x)
        x = torch.transpose(x, 0, 1)
        all_outputs, hidden = self.lstm(x, hidden)
        all_outputs = torch.transpose(all_outputs, 0, 1)
        last_seq_items = all_outputs[range(all_outputs.shape[0]), len_x]
        out = last_seq_items
        x = self.fc(out)
        return x, hidden

In [6]:
class GruSentimentClasifier(nn.Module):
    def __init__(
        self,
        embedding_dim,
        hidden_size,
        num_of_rnn_layers,
        out_size,
        emb_weights,
        bidirectional=False,
    ):
        super().__init__()
        self.num_of_rnn_layers = num_of_rnn_layers
        self.hidden_size = hidden_size
        if bidirectional:
            self.bidirectional = 2
        else:
            self.bidirectional = 1
        self.embeddings = nn.Embedding.from_pretrained(emb_weights)
        self.embeddings.requires_grad = False
        self.gru = nn.GRU(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            num_layers=num_of_rnn_layers,
            bidirectional=bidirectional,
            batch_first=False,
        )
        self.fc = nn.Linear(hidden_size * self.bidirectional, out_size)

    def init_hidden(self, batch_size):
        hidden = torch.zeros(
            self.num_of_rnn_layers * self.bidirectional, batch_size, self.hidden_size
        )
        state = torch.zeros(
            self.num_of_rnn_layers * self.bidirectional, batch_size, self.hidden_size
        )
        return hidden, state

    def forward(self, x, len_x, hidden):
        x = self.embeddings(x)
        x = torch.transpose(x, 0, 1)
        all_outputs, hidden = self.gru(x, hidden)
        all_outputs = torch.transpose(all_outputs, 0, 1)
        last_seq_items = all_outputs[range(all_outputs.shape[0]), len_x]
        out = last_seq_items
        x = self.fc(out)
        return x, hidden

# Training

In [59]:
model = LstmSentimentClasifier(100, 500, 2, 2, emb_weights, bidirectional=True)

optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
loss_fun = nn.CrossEntropyLoss()
model.train()

EPOCHS = 5

progress_bar = tqdm.tqdm(
                range(EPOCHS), total=EPOCHS, desc="Epoch"
            )

for epoch in progress_bar:
    losses = 0
    batches = 0
    for x, targets, len_x in train_loader:
        x = x.to(device)
        targets = targets.to(device)
        hidden, state = model.init_hidden(x.size(0))
        hidden, state = hidden.to(device), state.to(device)
        preds, _ = model(x, len_x, (hidden,state))
        preds = preds.squeeze(1)
        optimizer.zero_grad()
        loss = loss_fun(preds, targets)
        loss.backward()
        optimizer.step()
        losses += loss.item()
        batches +=1
    progress_bar.set_postfix({"Train loss": losses/batches})

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]


RuntimeError: CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
